In [1]:
import sys
sys.path.append('/workspace/Documents')
import CMR_HFpEF_Analysis.Defaults as Defaults
import CMR_HFpEF_Analysis.functions_collection as ff
import CMR_HFpEF_Analysis.Image_utils as util

import os
import numpy as np
import nibabel as nb
import pandas as pd
import re
import math
from datetime import datetime

main_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data/Patient_list'

# extract the radiology report for each patient

In [77]:
patient_list = pd.read_excel(os.path.join(main_path, 'AccessionNumber_ID_list.xlsx'))
print(patient_list.shape)

with open(os.path.join(main_path, 'QL033_20180802_221352_Rad.txt'), 'r') as file:
    # Read the entire contents of the file
    rad_reports = file.read()

with open(os.path.join(main_path, 'QL033_20180802_221352_Car.txt'), 'r') as file:
    # Read the entire contents of the file
    car_reports = file.read()

(1920, 21)


main script:

In [140]:
###################### main script
LVEF_list = []
for no in range(0,patient_list.shape[0]):
    print(no)
    accession = str(patient_list['AccessionNumber'].iloc[no])
    mrn = str(patient_list['PatientID(MRN)'].iloc[no])
    ourID = patient_list['OurID'].iloc[no]

    report_content = find_individual_report(rad_reports, accession, mrn, ourID)
    if len(report_content) == 1:
        report_content = find_individual_report(car_reports, accession, mrn, ourID)

        if len(report_content) == 1: # neither in cardiology nor in radiology:
            print('this one we couldn not find the report')
            LVEF_list.append([ourID, mrn, accession, 'no', 0])
            continue

    
    LVEF = find_LVEF(report_content)


    LVEF_list.append([ourID, mrn, accession, 'yes', LVEF])

    df = pd.DataFrame(LVEF_list, columns=['OurID', 'PatientID(MRN)', 'AccessionNumber', 'Find report?', 'LVEF'])
    df.to_excel(os.path.join(main_path, 'LVEF_list.xlsx'),index = False)



In [99]:
def find_individual_report(report, accession, mrn, ourID):

    lines = rad_reports.splitlines()

    # Initialize a list to store the line numbers where the sequence is found
    matching_lines_numbers = []
    # matching_lines_contents =[]

    # Iterate over each line and check if the sequence is present
    for i, line in enumerate(lines):
        if accession in line and mrn in line:
            matching_lines_numbers.append(i)  # Adding 1 to adjust for 0-based indexing
            # matching_lines_contents.append(line)
    
    if len(matching_lines_numbers) == 0:
        return ['']

    # write the report for this patient
    report_content = []; count = 0
    while True:
        l = lines[matching_lines_numbers[0] +count ]
        report_content.append(l)
        if 'report_end' in l:
            break
        if count > 500:
            break
        count += 1
    
    if ourID < 10:
        ourID = '000' + str(ourID)
    elif ourID >= 10 and ourID < 100:
        ourID = '00' + str(ourID)
    elif ourID >= 100 and ourID<1000:
        ourID = '0' + str(ourID)
    else:
        ourID = str(ourID)

    filename = 'ID_' + ourID + '_AccessionNumber_' + accession + '.txt'
    with open(os.path.join(main_path,'individual_reports', filename), 'w') as file:
        # Iterate over the list and write each element to a new line
        for item in report_content:
            file.write(item + '\n')
    
    return report_content



In [137]:
def find_LVEF(report_content):
    # find line that has LVEF info
    LVEF_line_numbers = []; full = []
    for ii, line in enumerate(report_content):
        if 'LVEF' in line:
            LVEF_line_numbers.append(ii); full.append(0)
        if 'left ventricular ejection fraction' in line:
            LVEF_line_numbers.append(ii); full.append(1)

    if len(LVEF_line_numbers) == 0:
        # no LVEF info
        return 0
   
    for ii in range(0,len(LVEF_line_numbers)):
        LVEF_line = report_content[LVEF_line_numbers[ii]]

        after = 0; ahead = 0; next_line = 0; LVEF = 0

        if full[ii] == 0:
            indexes = LVEF_line.find('LVEF')
            indexes = indexes + 3
        else: 
            indexes = LVEF_line.find('left ventricular ejection fraction')
            indexes = indexes + 33

        # after the key word
        for n in range(indexes, len(LVEF_line)):
            if '0' == LVEF_line[n] or '1' == LVEF_line[n] or '2' == LVEF_line[n] or '3' == LVEF_line[n] or '4' == LVEF_line[n] or '5' == LVEF_line[n] or '6' == LVEF_line[n] or '7' == LVEF_line[n] or '8' == LVEF_line[n] or '9' == LVEF_line[n]:
                LVEF_letters = []
                nn = 0
                while True:
                    if n + nn == len(LVEF_line):
                        break
                    if LVEF_line[n + nn] == ' ' or LVEF_line[n + nn] == '%' or LVEF_line[n + nn] == '-' or LVEF_line[n + nn] == '~':
                        break
                    LVEF_letters.append(LVEF_line[n+nn])
                    nn += 1
                LVEF = float(LVEF_line[n : n + len(LVEF_letters)]); after = 1
                break

        # ahead of the key word
        if after == 0:
            for n in np.arange(indexes, indexes-8, -1):
                if '0' == LVEF_line[n] or '1' == LVEF_line[n] or '2' == LVEF_line[n] or '3' == LVEF_line[n] or '4' == LVEF_line[n] or '5' == LVEF_line[n] or '6' == LVEF_line[n] or '7' == LVEF_line[n] or '8' == LVEF_line[n] or '9' == LVEF_line[n]:
                    LVEF_letters = []
                    nn = 0
                    while True:
                        if n - nn < 0:
                            break
                        if LVEF_line[n - nn] == ' ' or LVEF_line[n - nn] == '%' or LVEF_line[n - nn] == '-' or LVEF_line[n - nn] == '~':
                            break
                        LVEF_letters.append(LVEF_line[n - nn])
                        nn += 1
                    LVEF = float(LVEF_line[n - len(LVEF_letters) : n ]); ahead = 1
                    break

        # next line
        if after == 0 and ahead == 0:
            LVEF_line_next = report_content[LVEF_line_numbers[ii] + 1]
            for n in range(0,5):
                if n>=len(LVEF_line_next):
                    break
                if '0' == LVEF_line_next[n] or '1' == LVEF_line_next[n] or '2' == LVEF_line_next[n] or '3' == LVEF_line_next[n] or '4' == LVEF_line_next[n] or '5' == LVEF_line_next[n] or '6' == LVEF_line_next[n] or '7' == LVEF_line_next[n] or '8' == LVEF_line_next[n] or '9' == LVEF_line_next[n]:
                    LVEF_letters = []
                    nn = 0
                    while True:
                        if n + nn == 10:
                            break
                        if LVEF_line_next[n + nn] == ' ' or LVEF_line_next[n + nn] == '%' or LVEF_line_next[n + nn] == '-' or LVEF_line_next[n + nn] == '~':
                            break
                        LVEF_letters.append(LVEF_line_next[n + nn])
                        nn += 1
                    LVEF = float(LVEF_line_next[n: n+ len(LVEF_letters) ]); next_line = 1
                    break

        if LVEF != 0:
            return LVEF
        
    if LVEF == 0: # no numerical value of LVEF:
        return 0

# Drop duplicated cases:

In [3]:
df = pd.read_excel(os.path.join(main_path, 'HF_Patient_list_w_duplicated_images.xlsx'))
print(df.shape)
df = df.drop_duplicates(subset='AccessionNumber')
print(df.shape)

df.to_excel(os.path.join(main_path, 'HF_Patient_list.xlsx'), index=False)

(1920, 21)
(1418, 21)


# find out the unique patient and all his/her scans

In [9]:
data = pd.read_excel(os.path.join(main_path, 'HF_Patient_list_no_duplicated_images.xlsx'))

In [29]:
# each unique MRN is a unique patient
unique_mrn = []
unique_row_index = []

Result = []
for i in range(0, data.shape[0]):
    case = data.iloc[i]
    mrn = case['PatientID(MRN)']

    if mrn in unique_mrn:
        continue
    else:
        unique_mrn.append(mrn)

    # rows with the same mrn:
    rows = data[data['PatientID(MRN)'] == mrn]
 
    if rows.shape == 1:
        unique_row_index.append(i)
        dates_record = np.nan
    else:
        # this patient has more than 1 scans. let's pick the first one
        sorted_rows = rows.sort_values(by='StudyDate')
       
        earliest_scan_index = data.loc[data['OurID'] == sorted_rows.iloc[0]['OurID']].index[0]
        unique_row_index.append(earliest_scan_index)
        
        # then let's record the days between scans
        dates = sorted_rows['StudyDate']
        dates_record = []
        for d in range(1,dates.shape[0] ):
            date1 = datetime.strptime(str(dates.iloc[d-1]), '%Y%m%d')
            date2 = datetime.strptime(str(dates.iloc[d]), '%Y%m%d')
            dates_record.append((date2 - date1).days)
        
    Result.append([sorted_rows.iloc[0]['OurID'], rows.shape[0], dates_record])


df = pd.DataFrame(Result, columns = ['OurID', 'how_many_scans', 'days_between_scans'])

unique_list = data.iloc[unique_row_index]

# merge:
merged_df = unique_list.merge(df, on='OurID', how = 'left')
merged_df.to_excel(os.path.join(main_path, 'HF_Patient_list_unique_patient.xlsx'),index = False)


# Merge patient list with re-admission info

In [5]:
def convert_to_str(value):
    if isinstance(value, int) or (isinstance(value, str) and value.isdigit()):
        return str(value)
    return value

In [9]:
patient_list = pd.read_excel(os.path.join(main_path, 'Important_HFpEF_Patient_list_unique_patient_w_notes.xlsx'))
patient_list['AccessionNumber'] = patient_list['AccessionNumber'].apply(convert_to_str)

re_p_1yr = pd.read_excel(os.path.join(main_path, 're-admission-info/readmission_P_1yr.xlsx'))
re_p_1yr  = re_p_1yr.loc[:, ['AccessionNumber', 'flagReAdmissionWithinHalfYear']].copy()
re_p_1yr.rename(columns = {'flagReAdmissionWithinHalfYear': 'Readmission_Primary_1yr'}, inplace = True)
re_p_1yr['AccessionNumber'] = re_p_1yr['AccessionNumber'].apply(convert_to_str)

re_ps_1yr = pd.read_excel(os.path.join(main_path, 're-admission-info/readmission_P+S_1yr.xlsx'))
re_ps_1yr  = re_ps_1yr.loc[:, ['AccessionNumber', 'flagReAdmissionWithinHalfYear']].copy()
re_ps_1yr.rename(columns = {'flagReAdmissionWithinHalfYear': 'Readmission_Primary+Secondary_1yr'}, inplace = True)
re_ps_1yr['AccessionNumber'] = re_ps_1yr['AccessionNumber'].apply(convert_to_str)

re = pd.merge(re_p_1yr, re_ps_1yr, on='AccessionNumber', how = 'inner')

final = pd.merge(patient_list, re, on='AccessionNumber', how = 'inner')

# also add the Readmission date and department
re_ps_1yr = pd.read_excel(os.path.join(main_path, 're-admission-info/readmission_P+S_1yr.xlsx'))
info = re_ps_1yr.loc[:, ['AccessionNumber', 'ReAdmissionDTS', 'DepartmentDSC', 'ADTEventTypeDSC']].copy()
info['AccessionNumber'] = info['AccessionNumber'].apply(convert_to_str)

final = pd.merge(final, info, on='AccessionNumber', how = 'inner')

final.to_excel(os.path.join(main_path, 'Important_HFpEF_Patient_list_unique_patient_w_readmission.xlsx'),index = False)

# Merge HFpEF patient list with raw_data_correspondence

In [4]:
patient_list = pd.read_excel(os.path.join(main_path, 'Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx'))
raw_correspondence = pd.read_excel(os.path.join(main_path, 'raw_data_correspondence.xlsx'))

In [7]:
result = []
count = 0
for i in range(0,patient_list.shape[0]):
    ourID = patient_list['OurID'].iloc[i]

    # find this in raw_correrspondence
    raw = raw_correspondence.loc[raw_correspondence['OurID'] == ourID]
    folder_path = raw['folder_path'].iloc[0]

    if len(str(folder_path))<5:
        count += 1
        have_raw_data = 'no'
    else:
        have_raw_data = 'yes'

    result.append([ourID, patient_list['AccessionNumber'].iloc[i], have_raw_data, folder_path])

df = pd.DataFrame(result, columns = ['OurID', 'AccessionNumber','have_raw_data', 'folder_path'])
df.to_excel(os.path.join(main_path, 'raw_data_correspondence_HFpEF.xlsx'),index = False)